In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from conditional_inference.base import ModelBase, ResultsBase
from conditional_inference.stats import truncnorm, quantile_unbiased
from conditional_inference.utils import compute_projection_quantile, compute_projection_rvs
from conditional_inference.squ import SQU
from conditional_inference.rqu import RQU
from scipy.stats import multivariate_normal, norm
import seaborn as sns
import pandas as pd

sns.set()

x = np.array([3.3, 4.1, 4.2, 4.3, 6.2])
cov = np.array([
    [.01, 0, 0, 0, 0],
    [0, .25, 0, 0, 0],
    [0, 0, .05, 0, 0],
    [0, 0, 0, .05, 0],
    [0, 0, 0, 0, .05]
])

# x = np.arange(0, 6, 2)
# cov = np.identity(3)
# x

In [ ]:
import statsmodels.api as sm
df = pd.read_csv("../../mega-analysis/data/processed/effort_experiment.csv")
results = sm.OLS(df.y, pd.get_dummies(df.arm)).fit().get_robustcov_results()

In [ ]:
class Ranker(ModelBase):
    def fit(self, marginal=True, **kwargs):
        return MarginalResults(self, **kwargs) if marginal else SimultaneousResults(self, **kwargs)

class RankerResultsBase(ResultsBase):
    TWO_TAILED, LOWER_TAILED, UPPER_TAILED = ("two", "lower", "upper")

    @property
    def tails(self):
        return self._tails if hasattr(self, "_tails") else self.TWO_TAILED

    @tails.setter
    def tails(self, tails):
        self._tails = self._check_tails(tails)

    def _check_tails(self, tails):
        if tails is None:
            return self.tails

        if tails not in (None, self.TWO_TAILED, self.LOWER_TAILED, self.UPPER_TAILED):
            raise ValueError(f"tails must be one of {self.TWO_TAILED, self.LOWER_TAILED, self.UPPER_TAILED}, got {tails}.")
        
        return tails

    @staticmethod
    def _compute_delta_mean(mean, i):
        return np.delete(mean[i] - mean, i)

    @staticmethod
    def _compute_delta_cov(cov, i, j):
        repeat_i = np.repeat(np.atleast_2d(cov[i]), cov.shape[0], axis=0)
        repeat_j = np.repeat(np.atleast_2d(cov[j]), cov.shape[0], axis=0).T
        delta_cov = cov[i, j] + cov - repeat_i - repeat_j
        return np.delete(np.delete(delta_cov, i, axis=0), j, axis=1)

    @staticmethod
    def _stepdown(mean, cov, rvs, alpha):
        rejected, newly_rejected = np.full(len(mean), False), None
        std_deviation = np.sqrt(cov.diagonal())
        while newly_rejected is None or (newly_rejected.any() and (~rejected).any()):
            projection_quantile = np.quantile(rvs[:, ~rejected].max(axis=1), 1 - alpha)
            newly_rejected = (mean - projection_quantile * std_deviation > 0) & ~rejected
            rejected = newly_rejected | rejected

        return rejected


class MarginalResults(RankerResultsBase):
    def __init__(self, model, *args, tails="two", n_samples=10000, **kwargs):
        super().__init__(model, *args, **kwargs)
        mean = self.model.mean[self.indices]
        cov = self.model.cov[self.indices][:, self.indices]
        self.params = (-mean).argsort().argsort()
        self.pvalues = np.full(self.n_policies, np.nan)
        self.tails = tails

        self.delta_mean, self.delta_cov, self.rvs = [], [], []
        for i in range(self.n_policies):
            delta_mean = self._compute_delta_mean(mean, i)
            self.delta_mean.append(np.concatenate([delta_mean, -delta_mean]))
            delta_cov = self._compute_delta_cov(cov, i, i)
            delta_cov = np.vstack([
                np.hstack([delta_cov, -delta_cov]),
                np.hstack([-delta_cov, delta_cov])
            ])
            self.delta_cov.append(delta_cov)
            rvs = multivariate_normal.rvs(np.zeros(delta_cov.shape[0]), delta_cov, size=n_samples)
            self.rvs.append(rvs / np.sqrt(delta_cov.diagonal()))

    def compute_hypothesis_matrix(self, alpha=.05, tails=None):
        tails = self._check_tails(tails)
        matrix = []
        for i, (delta_mean, delta_cov, rvs) in enumerate(zip(self.delta_mean, self.delta_cov, self.rvs)):
            if tails != self.TWO_TAILED:
                # select the relevant objects for 1-tailed hypotheses
                indices = slice(0, self.n_policies-1) if tails == self.LOWER_TAILED else slice(self.n_policies-1, 2*(self.n_policies-1))
                delta_mean = delta_mean[indices]
                delta_cov = delta_cov[indices, indices]
                rvs = rvs[:, indices]

            rejected = self._stepdown(delta_mean, delta_cov, rvs, alpha)[:self.n_policies-1]
            rejected = np.insert(rejected, i, False)
            matrix.append(rejected)

        return pd.DataFrame(np.array(matrix).T, columns=self.model.exog_names, index=self.model.exog_names)

    def conf_int(self, alpha=.05, tails=None):
        tails = self._check_tails(tails)
        conf_int = super().conf_int(alpha)
        hypothesis_matrix = self.compute_hypothesis_matrix(alpha, tails)
        conf_int = np.array([hypothesis_matrix.sum(1), ranker.n_policies - hypothesis_matrix.sum(0) - 1]).T
        if tails == self.LOWER_TAILED:
            conf_int[:, 0] = 0
        elif tails == self.UPPER_TAILED:
            conf_int[:, 0] = self.n_policies - conf_int[:, 1] - 1
            conf_int[:, 1] = self.n_policies - 1
        return conf_int


# ranker = Ranker.from_csv("../../mega-analysis/data/conventional_estimates/penn_medicine.csv").fit(cols="sorted", marginal=True)
# ranker = Ranker.from_results(results).fit(cols="sorted", tails="lower", marginal=True)
ranker = Ranker(x, cov).fit(tails="two", marginal=True)
hypothesis_matrix = ranker.compute_hypothesis_matrix()
hypothesis_matrix
# ranker.point_plot(alpha=.2)
# rvs.shape, rejected.shape
# ranker[0].shape, ranker[1].shape, ranker[2].shape
# ranker.point_plot(alpha=.2)
# plt.show()

In [ ]:
ranker.point_plot()
plt.show()

In [ ]:
from itertools import combinations

class SimultaneousResults(RankerResultsBase):
    def __init__(self, model, *args, tails=None, n_samples=10000, **kwargs):
        super().__init__(model, *args, **kwargs)
        mean = model.mean[self.indices]
        cov = model.cov[self.indices][:, self.indices]
        self.params = (-mean).argsort().argsort()
        self.pvalues = np.full(self.n_policies, np.nan)
        self.tails = tails
        
        delta_mean, delta_cov = [], []
        for i in range(self.n_policies):
            delta_mean.append(self._compute_delta_mean(mean, i))
            delta_cov.append(np.hstack([self._compute_delta_cov(cov, i, j) for j in range(self.n_policies)]))
        self.delta_mean = np.concatenate(delta_mean)
        self.delta_cov = np.vstack(delta_cov)
        self.rvs = multivariate_normal.rvs(np.zeros(len(self.delta_mean)), self.delta_cov, size=n_samples)
        self.rvs /= np.sqrt(self.delta_cov.diagonal())

        # compute test statistics for the tau-best policies
        # test_stats[tau, i] is the test statistic for policy i when computing policies with rank <= tau
        arr = self.delta_mean / np.sqrt(self.delta_cov.diagonal())
        arr = arr.reshape((self.n_policies, self.n_policies - 1))
        arr = np.hstack((arr, np.zeros((self.n_policies, 1))))
        arr.sort()
        self.test_stats = -arr.T

    def _get_mask(self, arr, get_mask_for_policy):
        if isinstance(arr, int):
            return get_mask_for_policy(arr)
        elif len(arr) == 0:
            return np.full(len(self.delta_mean), False)
        return np.array([get_mask_for_policy(i) for i in arr]).any(axis=0)

    def _get_i_minus_j_mask(self, arr):
        def get_mask_for_policy(i):
            indices = np.arange(i * (self.n_policies - 1), (i + 1) * (self.n_policies - 1))
            mask = np.zeros(len(self.delta_mean))
            mask[indices] = 1
            return mask.astype(bool)

        return self._get_mask(arr, get_mask_for_policy)

    def _get_j_minus_i_mask(self, arr):
        def get_mask_for_policy(i):
            # delta_ji are the i-1'th indicies in groups before start and the i'th indicies in the groups after end
            start, end = i * (self.n_policies - 1), (i + 1) * (self.n_policies - 1)
            indices = np.concatenate((np.arange(i - 1, start, self.n_policies - 1), np.arange(end + i, len(self.delta_mean), self.n_policies - 1)))
            if i == 0:
                # indices will start with -1, which is incorrect
                indices = indices[1:]
            mask = np.zeros(len(self.delta_mean))
            mask[indices] = 1
            return mask.astype(bool)

        return self._get_mask(arr, get_mask_for_policy)

    def compute_best_policies(self, tau=0, alpha=.05, superset=True):
        def compute_critical_value(subset):
            k_mask = ~self._get_i_minus_j_mask(subset)
            critical_value = np.quantile(self.rvs[:, k_mask & ~rejected_mask].max(axis=1), 1 - alpha)
            return critical_value

        if tau == self.n_policies - 1:
            return np.full(self.n_policies, True)

        if superset:
            test_stats = self.test_stats[tau]
        else:
            test_stats = -self.test_stats[tau + 1]
            tau = self.n_policies - tau - 1
            
        rejected, newly_rejected = np.full(self.n_policies, False), None
        while newly_rejected is None or (newly_rejected.any() and (~rejected).any()):
            rejected_mask = self._get_j_minus_i_mask(np.where(rejected)[0])
            critical_value = max([compute_critical_value(i) for i in combinations(np.arange(self.n_policies), tau)])
            newly_rejected = (test_stats > critical_value) & ~rejected
            rejected = rejected | newly_rejected

        return ~rejected if superset else rejected

    def compute_hypothesis_matrix(self, alpha=.05):
        matrix = []
        rejected = self._stepdown(self.delta_mean, self.delta_cov, self.rvs, alpha)
        for i in range(self.n_policies):
            rejected_i = rejected[self._get_i_minus_j_mask(i)]
            rejected_i = np.insert(rejected_i, i, False)
            matrix.append(rejected_i)

        return pd.DataFrame(np.array(matrix).T, columns=self.model.exog_names, index=self.model.exog_names)

    def conf_int(self, alpha=.05):
        hypothesis_matrix = self.compute_hypothesis_matrix(alpha)
        return np.array([hypothesis_matrix.sum(1), ranker.n_policies - hypothesis_matrix.sum(0) - 1]).T

ranker = Ranker(x, cov).fit(marginal=False)
# ranker = Ranker.from_csv("../../mega-analysis/data/conventional_estimates/walmart.csv").fit(cols="sorted", marginal=False)
# ranker = Ranker.from_results(results).fit(marginal=False)
# ranker.compute_hypothesis_matrix()
# x = 2 * np.arange(3)
# cov = np.identity(3)
# ranker = Ranker(x, cov).fit(cols="sorted", marginal=False)
# ranker._get_j_minus_i_indices(0)
ranker.point_plot()
plt.show()

In [ ]:
ranker.compute_best_policies(tau=4, superset=False)

In [ ]:
ranker.compute_hypothesis_matrix()

In [ ]:
best_policies = ranker.compute_best_policies(0, .5)
best_policies, best_policies.sum()

In [ ]:
ranker.delta_mean, ranker.delta_mean.reshape((ranker.n_policies, ranker.n_policies - 1))

In [ ]:
arr = ranker.delta_mean / np.sqrt(ranker.delta_cov.diagonal())
arr = arr.reshape((ranker.n_policies, ranker.n_policies - 1))
arr = np.hstack((arr, np.zeros((ranker.n_policies, 1))))
arr.sort()
ranker.test_stats = -arr
ranker.test_stats

In [ ]:
from itertools import combinations

In [ ]:
def compute_best_tau(tau=0, alpha=.05):
    def compute_critical_value(subset):
        k_mask = ~ranker._get_i_minus_j_mask(subset)
        critical_value = np.quantile(ranker.rvs[:, k_mask & ~rejected_mask].max(axis=1), 1 - alpha)
        return critical_value

    test_stats = ranker.test_stats[:, tau]
    rejected, newly_rejected = np.full(ranker.n_policies, False), None
    while newly_rejected is None or (newly_rejected.any() and (~rejected).any()):
        rejected_mask = ranker._get_j_minus_i_mask(np.where(rejected)[0])
        if tau == ranker.n_policies:
            critical_value = 0
        else:
            critical_value = max([compute_critical_value(i) for i in combinations(np.arange(ranker.n_policies), tau)])
        newly_rejected = (test_stats > critical_value) & ~rejected
        rejected = rejected | newly_rejected

    return rejected

compute_best_tau(0, .01)